In [2]:
import os
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
def recall_cnn(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_cnn(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_cnn(y_true, y_pred):
    precision = precision_cnn(y_true, y_pred)
    recall = recall_cnn(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


In [ ]:
METRICS = ["accuracy", recall_cnn, precision_cnn, f1_cnn]

In [ ]:
def visualize_history(history):
    #  "Plot Accuracy"
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

    # "Plot Loss"
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("model loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


In [ ]:
def Train_CNN_Model(epochs_num, model, train_iter, valid_iter, export_dir="./export", name="default"):
    # -------------------------------------------------------------------------
    #                        Train CNN Model
    # -------------------------------------------------------------------------

    epochs = epochs_num

    es = EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, patience=40)

    mc = ModelCheckpoint(f"{export_dir}/model_{name}.h5", monitor="val_accuracy", mode="max", save_best_only=True)

    history = model.fit(
        train_iter,
        steps_per_epoch=len(train_iter),
        validation_data=valid_iter,
        validation_steps=len(valid_iter),
        epochs=epochs,
        verbose=1,
        callbacks=[mc, es],
    )

    return history

In [3]:
def VGG16(input_shape, num_classes):
    model = Sequential()
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding="same",
            input_shape=input_shape,
            name="block1_conv1",
            activation="relu",
        )
    )
    model.add(BatchNormalization(name="bn1"))

    model.add(
        Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block1_conv2", activation="relu")
    )
    model.add(BatchNormalization(name="bn2"))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="block1_pool"))

    model.add(
        Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block2_conv1", activation="relu")
    )
    model.add(BatchNormalization(name="bn3"))

    model.add(
        Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block2_conv2", activation="relu")
    )
    model.add(BatchNormalization(name="bn4"))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="block2_pool"))

    model.add(
        Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv1", activation="relu")
    )
    model.add(BatchNormalization(name="bn5"))

    model.add(
        Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv2", activation="relu")
    )
    model.add(BatchNormalization(name="bn6"))

    model.add(
        Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block3_conv3", activation="relu")
    )
    model.add(BatchNormalization(name="bn7"))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="block3_pool"))
    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv1", activation="relu")
    )
    model.add(BatchNormalization(name="bn8"))

    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv2", activation="relu")
    )
    model.add(BatchNormalization(name="bn9"))

    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block4_conv3", activation="relu")
    )
    model.add(BatchNormalization(name="bn10"))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="block4_pool"))
    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block5_conv1", activation="relu")
    )
    model.add(BatchNormalization(name="bn11"))

    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block5_conv2", activation="relu")
    )
    model.add(BatchNormalization(name="bn12"))

    model.add(
        Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", name="block5_conv3", activation="relu")
    )
    model.add(BatchNormalization(name="bn13"))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="block5_pool"))

    model.add(Flatten(name="flatten"))
    model.add(Dense(units=4096, activation="relu", name="fc1"))
    model.add(Dense(units=4096, activation="relu", name="fc2"))
    model.add(Dense(units=num_classes, activation="softmax", name="predictions"))

    return model


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 64, 64, 256)       2

In [ ]:
input_shape = 224, 224, 3
model_VGG16 = VGG16(input_shape, 8)
model_VGG16.summary()

In [4]:
data_folder = 'data/Flowers'
train_data_folder = 'data/train'
validation_data_folder = 'data/validation'
flower_types = ["Babi", "Calimerio", "Chrysanthemum", "Hydrangeas", "Lisianthus", "Pingpong", "Rosy", "Tana"]

# Create empty lists to store image paths and corresponding labels
train_image_paths = []
train_labels = []
val_image_paths = []
val_labels = []

# Loop over the flower types and add image paths and labels to the lists
# for i, flower_type in enumerate(flower_types):
#     folder_path = os.path.join(data_folder, flower_type)
#     files = os.listdir(folder_path)
#     random.shuffle(files)
#     split_index = int(0.8 * len(files))
#     train_files = files[:split_index]
#     val_files = files[split_index:]
    
#     for file_name in train_files:
#         if file_name.endswith('.jpg'):
#             image_path = os.path.join(folder_path, file_name)
#             train_image_paths.append(image_path)
#             train_labels.append(i)
            
#             os.makedirs(os.path.join(train_data_folder, flower_type), exist_ok=True)
#             shutil.copy(image_path, os.path.join(train_data_folder, flower_type, file_name))
            
#     for file_name in val_files:
#         if file_name.endswith('.jpg'):
#             image_path = os.path.join(folder_path, file_name)
#             val_image_paths.append(image_path)
#             val_labels.append(i)
            
#             os.makedirs(os.path.join(validation_data_folder, flower_type), exist_ok=True)
#             shutil.copy(image_path, os.path.join(validation_data_folder, flower_type, file_name))
            
# Define the data generators for the training and validation sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_data_folder,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_folder,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 3693 images belonging to 8 classes.
Found 928 images belonging to 8 classes.


In [5]:
train_features = vgg_model.predict(train_generator,verbose=1)
val_features = vgg_model.predict(validation_generator,verbose=1)

 6/58 [==>...........................] - ETA: 6:27

In [11]:
adam = Adam(learning_rate=0.00006, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
model_VGG16.compile(loss="binary_crossentropy", optimizer=adam, metrics=METRICS)
history_vgg = Train_CNN_Model(50, model_VGG16, train_generator, validation_generator, export_dir="./export", name="vgg_t1")

In [ ]:
visualize_history(history_vgg)

ResNet

In [ ]:
# Resnet-50 architecture
"""We will re-built the Resnet-50 architecture with reference from []()"""

"""First block is the initial block for Layer0 consists of a size of 7*7 kernel and 0 padding,
Since x_shortcut are 3 matrixes, and we can only add input and output if they are in the same shape.
Therefore, if the convolution + Batch Normalization operation allows to do so, we will take use of it in the first and following block
At the end of the block we will create 3*3 max pooling to reduce the dimension of the input."""


def initial_block(Input, filters, stride=1, size=7):
    x = Conv2D(filters, kernel_size=(size, size), strides=(stride, stride), padding="same")(Input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    return x


"""The next block is call the convotional expand block, this block is expand from original convolutional block.
In this block, first we create the convolution 2D with kernel size of 3x3, with 64 filter (kernels) and at last 1x1,
with 256 kernel
These three layers are repeated in total 3 time so giving us 9 layers in this step.
In expand convolution bottleneck block, the usage of projection shortcut will be introduced. The projection shortcut is used to match dimensions (done by 1x1 convolutions).
The shortcuts go across feature maps of two sizes, they are performed with a stride of 2."""


def conv_exp_bottleneck_block(Input, filters, k_size, k2_size, stride=1):
    # Contracting 1*1 conv
    x = Conv2D(filters, kernel_size=(k2_size, k2_size), strides=(stride, stride), padding="same")(Input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Depth preserving 3*3 conv
    x = Conv2D(filters, kernel_size=(k_size, k_size), strides=(1, 1), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Expanding 1*1 Conv
    x = Conv2D(filters * 4, kernel_size=(k2_size, k2_size), strides=(1, 1), padding="same")(x)
    x = BatchNormalization()(x)

    # Projection shortcut
    skip_conv = Conv2D(filters * 4, kernel_size=(k2_size, k2_size), strides=(stride, stride), padding="same")(Input)
    skip = BatchNormalization()(skip_conv)

    # Skip connection
    x = Add()([x, skip])

    return x


"""Normal Convolutional bottleneck block will containfirst conv block of kernel of size 1x1, 128 kernels
After that a conv block of a kernel of 3x3, with 128 kernels and
at last we expand conv block a kernel of 1x1, with 512 kernels. In this function, we eliminate project shortcut to reduce training time """


def convnorm_bottleneck_block(Input, filters, k_size, k2_size, stride=1):
    # Contracting 1*1 conv
    x = Conv2D(filters, kernel_size=(k2_size, k2_size), strides=(stride, stride), padding="same")(Input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Depth preserving 3*3 Conv
    x = Conv2D(filters, kernel_size=(k_size, k_size), strides=(stride, stride), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Expanding 1*1 Conv
    x = Conv2D(filters * 4, kernel_size=(k2_size, k2_size), strides=(stride, stride), padding="same")(x)
    x = BatchNormalization()(x)

    # Identity skip connection
    x = Add()([x, Input])

    return x


"""This function is responsible for building a complete resnet50 with deep bottleneck architecture.
First input shape must be changed to match with the iterator image size of 27x27x3.
h,w will be input height and width.
no_of_outputs is the number of classes to learn from.
activation type will let we choose between different activation type.
initial value of first conv block stride is 2 and initial conv block will have kernel 7x7.
 """


def build_bottleneck_resnet(h, w, no_of_outputs, activation_type, first_conv_stride=2, first_conv_kernel_size=7):
    # Creating input tensor
    inputs = Input(shape=(h, w, 3), name="image_input")

    # STAGE 1:
    # Inital Conv block like paper architecture shown -> 1 layer
    x = initial_block(inputs, 64, first_conv_stride, first_conv_kernel_size)

    # STAGE 2
    # In the following stage, we make it little bit different, to take into account the usage of project shortcut.
    # To make the dimension compatible first.
    # Expanding block1 with projection shortcut. -> 3 layers
    x = conv_exp_bottleneck_block(x, 64, 3, 1, 1)
    x = Activation("relu")(x)

    # Repeating block of Conv1 Stage 2 - paper value is repeat 3 times. However, we create the conv expanding block in previous step
    # in this step, only repeat 2 times -> 6 layers
    for i in range(2):
        x = convnorm_bottleneck_block(x, 64, 3, 1, 1)
        x = Activation("relu")(x)

    # STAGE 3
    # Expanding block2 with projection shortcut, the reason is the same with stage 2 -> 3 layers
    x = conv_exp_bottleneck_block(x, 128, 3, 1, 2)
    x = Activation("relu")(x)

    # Repeating block of Conv2. Initial value with normal resnet architecture is 4. However, since the usage of
    # expanding convolution is used above, we reduce it to only 3 repitation. -> 9 layer
    for i in range(3):
        x = convnorm_bottleneck_block(x, 128, 3, 1)
        x = Activation("relu")(x)

    # STAGE 4
    # Expanding block3 with projection shortcut -> 3 layers
    x = conv_exp_bottleneck_block(x, 256, 3, 1, 2)
    x = Activation("relu")(x)

    # Repeating block of Conv3 -> 15 layers
    for i in range(5):
        x = convnorm_bottleneck_block(x, 256, 3, 1, 1)
        x = Activation("relu")(x)

    # STAGE 5
    # Expanding block4 with projection shortcut -> 3 layers
    x = conv_exp_bottleneck_block(x, 512, 3, 1, 2)
    x = Activation("relu")(x)

    # Repeating block of Conv4 ->  6 layers
    for i in range(2):
        x = convnorm_bottleneck_block(x, 512, 3, 1, 1)
        x = Activation("relu")(x)

    shape = K.int_shape(x)

    # Final Stage - 1 layers
    # Average pooling layer
    x = AveragePooling2D(pool_size=(shape[1], shape[2]), strides=(1, 1))(x)
    # x = GlobalAveragePooling2D()(x)

    # Classifier Block
    x = Flatten()(x)
    x = Dense(no_of_outputs, activation=activation_type)(x)

    model = Model(inputs=inputs, outputs=x)

    return model

In [ ]:
ResNet50_model = build_bottleneck_resnet(224, 224, 3, "binary_crossentropy", "sigmoid", 2, 7)
ResNet50_model.summary()

In [ ]:
% % time
adam = Adam(learning_rate=0.00006, beta_1=0.9,
            beta_2=0.999, epsilon=None, amsgrad=False)
ResNet50_model.compile(loss='binary_crossentropy',
                       optimizer=adam, metrics=METRICS)

In [ ]:
history_resnet50 = Train_CNN_Model(
    50, ResNet50_model, train_generator, validation_generator, export_dir="./export", name="resnet50_task1_final"
)
history_resnet50

In [ ]:
visualize_history(history_resnet50)

In [ ]:
# from PIL import Image
#
# def recommend_similar_images(vgg_model, image_path, data_folder, flower_types, top_n=10):
#     # Load and preprocess the input image
#     img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
#     x = tf.keras.preprocessing.image.img_to_array(img)
#     x = np.expand_dims(x, axis=0)
#     x = tf.keras.applications.vgg16.preprocess_input(x)
#
#     # Extract the feature vector of the input image
#     features = vgg_model.predict(x)
#
#     # Calculate cosine similarities between the input image and all the images in the dataset
#     similarities = []
#     for i, flower_type in enumerate(flower_types):
#         folder_path = os.path.join(data_folder, flower_type)
#         for file_name in os.listdir(folder_path):
#             if file_name.endswith('.jpg'):
#                 image_path = os.path.join(folder_path, file_name)
#                 img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
#                 x = tf.keras.preprocessing.image.img_to_array(img)
#                 x = np.expand_dims(x, axis=0)
#                 x = tf.keras.applications.vgg16.preprocess_input(x)
#                 features_i = vgg_model.predict(x)
#                 similarity = cosine_similarity(features, features_i)
#                 similarities.append((image_path, flower_type, similarity[0][0]))
#
#     # Sort the similarities in descending order and select the top_n images
#     similarities = sorted(similarities, key=lambda x: x[2], reverse=True)[:top_n]
#
#     # Display the input image, filename, and flower type
#     display(Image(filename=image_path))
#     print("Input Image: ", os.path.basename(image_path), flower_types[train_labels[image_paths.index(image_path)]])
#
#     # Display the top_n similar images with their filenames, flower types, and cosine similarity scores
#     for image_path, flower_type, similarity in similarities:
#         display(Image(filename=image_path))
#         print(os.path.basename(image_path), flower_type, similarity)

In [ ]:
# recommend_similar_images(vgg_model, 'data/Flowers/Babi/babi_1.jpg', 'data/Flowers', flower_types)